In [ ]:
!wget https://goo.gl/8hY5ER
!tar xf 8hY5ER

--2019-02-21 11:56:49--  https://goo.gl/8hY5ER
Loaded CA certificate '/etc/ssl/certs/ca-certificates.crt'
Resolving goo.gl (goo.gl)... 216.58.194.142, 2607:f8b0:4000:80d::200e
Connecting to goo.gl (goo.gl)|216.58.194.142|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://zenodo.org/record/1203745/files/UrbanSound8K.tar.gz [following]
--2019-02-21 11:56:50--  https://zenodo.org/record/1203745/files/UrbanSound8K.tar.gz
Resolving zenodo.org (zenodo.org)... 137.138.76.77
Connecting to zenodo.org (zenodo.org)|137.138.76.77|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6023741708 (5.6G) [application/octet-stream]
Saving to: '8hY5ER'

8hY5ER                6%[>                   ] 357.41M   225KB/s    eta 5h 3m  

In [ ]:
!ls UrbanSound8K/audio/

In [ ]:
import IPython
IPython.display.Audio("Ed Sheeran-Little Bird Lyrics.mp3")

In [ ]:
!mkdir 'UrbanSound8K/spectrogram'

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.pyplot import specgram
import librosa
import numpy as np
def create_fold_spectrograms(fold):
    spectrogram_path = Path('UrbanSound8K/spectrogram/')  
    audio_path = Path('UrbanSound8K/audio/')  
    print(f'Processing fold {fold}')
    os.mkdir(spectrogram_path/fold)
    for audio_file in list(Path(audio_path/f'fold{fold}').glob('*.wav')):
        samples, sample_rate = librosa.load(audio_file)
        fig = plt.figure(figsize=[0.72,0.72])
        ax = fig.add_subplot(111)
        ax.axes.get_xaxis().set_visible(False)
        ax.axes.get_yaxis().set_visible(False)
        ax.set_frame_on(False)
        filename  = spectrogram_path/fold/Path(audio_file).name.replace('.wav','.png')
        S = librosa.feature.melspectrogram(y=samples, sr=sample_rate)
        librosa.display.specshow(librosa.power_to_db(S, ref=np.max))
        plt.savefig(filename, dpi=400, bbox_inches='tight',pad_inches=0)
        plt.close('all')


In [ ]:
for i in range(1, 11):
    create_fold_spectrograms(str(i))

In [ ]:
!mkdir 'UrbanSound8K/data'

In [ ]:
import os
import glob
import shutil
from pathlib import Path  
data_path = Path('UrbanSound8K/data/') 
spectrogram_path = Path('UrbanSound8K/spectrogram/')  
def create_fold_directory(fold):
    png_files =  list(Path(spectrogram_path/fold).glob('*.png'))
    os.mkdir(data_path/fold)
    os.mkdir(data_path/fold/'train')
    os.mkdir(data_path/fold/'valid')
    for label in labels:
        os.mkdir(data_path/fold/'train'/label)
        os.mkdir(data_path/fold/'valid'/label)
     
    for file in png_files:
        label = file.as_posix().split('-')[1]
        shutil.copyfile(file, data_path/fold/'valid'/labels[int(label)]/file.name)   
    for i in range(1, 11):       
        if str(i) == fold:
            continue
        png_files =  list(Path(spectrogram_path/str(i)).glob('*.png'))
        for file in png_files:
            label = file.as_posix().split('-')[1]
            shutil.copyfile(file, data_path/fold/'train'/labels[int(label)]/file.name)

In [ ]:
for i in range(1, 11):
    create_fold_directory(str(i))

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
from fastai import *
from fastai.vision import *

In [ ]:
data_directory = Path('UrbanSound8K/data')
# don't use any transformations because it doesn't make sense in the case of a spectrogram
# i.e. flipping a spectrogram changes the meaning
data = ImageDataBunch.from_folder(data_directory/'1', ds_tfms=[], size=224)
data.normalize(imagenet_stats)

In [ ]:
data.show_batch(rows=6, figsize=(12,12))

In [ ]:
def process_fold(fold):
    data_directory = Path('UrbanSound8K/data')
    data = ImageDataBunch.from_folder(data_directory/fold, ds_tfms=[], size=224)
    data.normalize(imagenet_stats)
    learn = ConvLearner(data, models.resnet34, metrics=error_rate)
    learn.fit_one_cycle(3)
    learn.unfreeze()
    learn.fit_one_cycle(1)
    learn.fit_one_cycle(3, max_lr=slice(1e-6,1e-4))

In [ ]:
process_fold('1')

In [ ]:
process_fold('2')

In [ ]:
process_fold('3')

In [ ]:
process_fold('4')

In [ ]:
process_fold('5')

In [ ]:
process_fold('6')

In [ ]:
process_fold('7')

In [ ]:
process_fold('8')

In [ ]:
process_fold('9')

In [ ]:
process_fold('10')